# Tutorial: How to describe a set of images in terms of visual clusters

This is a tutorial on how to process a set of images with a pretrained computer vision model to obtain latent representations, cluster them with K-means, and calculate how many photos belong to each category. This example was adapted by the following [example](https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34) from Gabe Flomo. If you are having problems with the tutorial, or have any questions contact me at sebtop@kt.dtu.dk, [@littlecodelover](https://twitter.com/littlecodelover) or [LinkedIn](https://www.linkedin.com/in/sebastian-topalian-3b1743204/).





Here we use images related to the following publication(s):
- ECO-STP Conference: Quantitative Image Analysis of Biosolids: Evaluating morphological parameters versus visual features # Insert DOI here

- Full journal paper: Title # Insert DOI here

## 1. Change runtime type
Before running the code you should change the runtime to a runtime type with a GPU. This helps us speed up the inference from the image model used later on. In the menu above select runtime, change runtime type and then from the hardware accelerator drop down menu select GPU.

## 2. Download zip file and unzip

First we download pictures from a sample of inactivated biomass. These photos were taken and segmented with an Ocelloscope and accompanying software from [ParticleTech](https://particletech.dk).

In [1]:
!git clone https://github.com/waterboy96/ImageAnalysis.git

Cloning into 'ImageAnalysis'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 5.93 MiB | 6.32 MiB/s, done.


In [2]:
!unzip -q ImageAnalysis/Particles.zip

## 3. Import necessary packages

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from keras.applications import Xception
from keras.models import Model
from keras.utils import image_dataset_from_directory, load_img


from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## 4. Load Xception model and generate feature vectors

In [ ]:
model = Xception()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

54452224/91884032 [================>.............] - ETA: 1s

In [ ]:
ds = image_dataset_from_directory('Particles', batch_size= 64, image_size = (299,299), labels = None, shuffle = False, label_mode= None)
df = pd.DataFrame(model.predict(ds))

In [ ]:
df.head()

We now have a vector of length 2048 for each picture in our sample. These vectors represent the output of the penultimate layer in the Xception model which can be used to describe our images.

## 5. Reduce dimensionality with PCA

In [ ]:
df_scaled = (df-df.mean())/df.std()
df_scaled.dropna(axis = 1, inplace = True)

In [ ]:
pca = PCA()
pca.fit(df_scaled)

We pick a threshold of how much of the variance that should be retained. Here we select 99%. Reducing the dimensionality of the vectors helps with the computational time of clustering in the next step.

In [ ]:
variance_threshold = 0.99

plt.plot([x for x in range(len(pca.explained_variance_))], np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components', fontweight = 'bold')
plt.ylabel('Cumulative Variance Explained Ratio', fontweight = 'bold')
ax = plt.gca()
plt.title('Skree Plot for PCA on Xception Feature Vectors', fontweight = 'bold')
plt.axvline(np.argmax(np.cumsum(pca.explained_variance_ratio_)>variance_threshold), linestyle = '--', c = 'red')
plt.text(700,0.7,f'{variance_threshold*100}% Variance Threshold at {np.argmax(np.cumsum(pca.explained_variance_ratio_)>variance_threshold)}')
plt.show()

An argument could be made to reduce the threshold to the point where the curve evens out for further reduction.

In [ ]:
threshold = np.argmax(np.cumsum(pca.explained_variance_ratio_)>variance_threshold)

In [ ]:
pca_threshold = PCA(n_components=threshold)

In [ ]:
PCs = pca_threshold.fit_transform(df_scaled)

## 6. K-means clustering and cluster counting

In [ ]:
KM = KMeans(n_clusters = 6)

In [ ]:
KM.fit(PCs)

In [ ]:
labels = labels = KM.predict(PCs)

In [ ]:
def Count_values(X,Clusters):
    return pd.DataFrame([(len(X[X == i])/len(X)) for i in range(Clusters)]).T

In [ ]:
Count_values(labels,6)

We can see that the distribution between the different clusters for this sample is quite uneven. Cluster 0, 1 have above 5% while 4, 6, 11, 14 have below 0.01%.

This vector is what we in the beforementioned literature reffered to as cluster participation vectors which showed better performance for predicitng reject turbidity than both clusters built on conventional morphological parameters such as length, area etc. and also better performance than the mean of said morphological parameters.

## 7. Visualisation of clusters

In [ ]:
particles = []

with os.scandir('Particles')as files:
    for file in files:
        if file.name.endswith('.png'):
            particles.append(file.name)

In [ ]:
particles = sorted(['Particles/'+p for p in particles if not p.startswith('.')])

In [ ]:
groups = {}

for file, cluster in zip(particles, labels):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
def view_cluster(cluster):
    plt.figure(figsize = (25,25));
    # gets the lits of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
        
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10, index+1);
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

In [ ]:
view_cluster(1) 

Try changing the above number to visualise a different cluster. (Don't forget cluster 0)

# Conclusion

That is it for this tutorial. If you have a particularly large number of photos I would suggest considering down sampling as K-means scales poorly. For questions contact me at sebtop@kt.dtu.dk, [@littlecodelover](https://twitter.com/littlecodelover) or [LinkedIn](https://www.linkedin.com/in/sebastian-topalian-3b1743204/).

